In [1]:
from definitions import target_var, data_path, incl_countries, incl_years, donor_countries, treatment_countries, \
    country_col, year_col, month_col, quarter_col, date_col, model_val, timeframe_val, tables_path_res, save_output, \
    fake_num, figures_path_data, agg_val, interpolation_val
from helper_functions_general import read_data, select_country_year_measure, month_name_to_num, rename_order_scale, \
    downsample_month_to_quarter, quarter_to_month, upsample_quarter_to_month, get_timeframe_col, get_trans, \
    get_data_path, get_fig_path, get_impl_date, get_impl_date
from statistical_tests import stat_test
from plot_functions import plot_series

import SparseSC
import numpy as np

In [2]:
timeframe = 'm'
stat = 'stat'

In [3]:
df = read_data(source_path=get_data_path(timeframe=timeframe), file_name=f'total_{timeframe}')
df

df_log_diff = read_data(source_path=get_data_path(timeframe=timeframe), file_name=f'total_{timeframe}_{stat}')
df_log_diff

,country,date,year,month,co2,gdp,infl,pop,brent,co2_cap,gdp_cap
0,austria,2002-01-01,2002,1,0.052060,0.009157,-0.004,0.001714,-0.281373,0.069483,0.007443
1,austria,2002-02-01,2002,2,-0.025243,0.020299,-0.001,0.001568,-0.294420,-0.066421,0.018731
2,austria,2002-03-01,2002,3,0.018000,0.031382,-0.002,0.001422,0.081951,-0.005582,0.029961
3,austria,2002-04-01,2002,4,0.028786,0.042408,-0.009,0.001275,-0.117204,-0.092207,0.041132
4,austria,2002-05-01,2002,5,0.054459,0.047218,-0.013,0.001162,-0.130776,0.028971,0.046056
...,...,...,...,...,...,...,...,...,...,...,...
4061,united kingdom,2019-06-01,2019,6,-0.017357,0.020092,-99999.000,-0.000658,-0.620213,-0.034180,0.020749
4062,united kingdom,2019-07-01,2019,7,-0.034553,0.023307,-99999.000,-0.000707,-0.576092,-0.064917,0.024013
4063,united kingdom,2019-08-01,2019,8,-0.020662,0.025343,-99999.000,-0.000756,-0.544328,-0.037323,0.026098
4064,united kingdom,2019-09-01,2019,9,-0.005156,0.027362,-99999.000,-0.000805,-0.567650,0.000994,0.028167


In [4]:
df

,country,date,year,month,co2,gdp,infl,pop,brent,co2_cap,gdp_cap
0,austria,2000-01-01,2000,1,9.726928e+09,2.309478e+11,0.014,8.003000e+09,25.51,1.215410,28.857653
1,austria,2000-02-01,2000,2,8.532685e+09,2.321217e+11,0.019,8.004700e+09,27.78,1.065959,28.998180
2,austria,2000-03-01,2000,3,8.134567e+09,2.332957e+11,0.019,8.006400e+09,27.49,1.016008,29.138647
3,austria,2000-04-01,2000,4,6.546738e+09,2.344696e+11,0.018,8.008100e+09,22.76,0.817514,29.279055
4,austria,2000-05-01,2000,5,5.603262e+09,2.352176e+11,0.016,8.010100e+09,27.74,0.699525,29.365122
...,...,...,...,...,...,...,...,...,...,...,...
4517,united kingdom,2019-06-01,2019,6,2.864851e+10,3.302814e+12,-99999.000,6.684433e+10,64.22,0.428585,49.410534
4518,united kingdom,2019-07-01,2019,7,2.872032e+10,3.322513e+12,-99999.000,6.686800e+10,63.92,0.429508,49.687638
4519,united kingdom,2019-08-01,2019,8,2.809851e+10,3.338771e+12,-99999.000,6.689167e+10,59.04,0.420060,49.913109
4520,united kingdom,2019-09-01,2019,9,3.070698e+10,3.355029e+12,-99999.000,6.691533e+10,62.83,0.458893,50.138420


In [10]:
df[df['country']=='netherlands'].groupby(['year']).sum()['co2']

year
2000    1.817788e+11
2001    1.867306e+11
2002    1.877745e+11
2003    1.897458e+11
2004    1.931176e+11
2005    1.901200e+11
2006    1.860358e+11
2007    1.875129e+11
2008    1.899811e+11
2009    1.865765e+11
2010    1.985725e+11
2011    1.869015e+11
2012    1.863921e+11
2013    1.856265e+11
2014    1.779121e+11
2015    1.854119e+11
2016    1.863759e+11
2017    1.851158e+11
2018    1.801398e+11
2019    1.426723e+11
Name: co2, dtype: float64

In [9]:
treatment_country = 'united kingdom'
impl_date = get_impl_date(treatment_country)
impl_date

'2013-01-01'